In [1]:
#各社のTokenファイルを読み込み、データフレームに格納
import pandas as pd
import os
import fitz
import re
import datetime
import sys
sys.path.append('/work_dir') 
from sklearn.feature_extraction.text import TfidfVectorizer

# 読み込むcsvファイル名を記載したリストファイル指定
filename="token_list.txt"
#filename="token_list2.txt"
list_file= "output/IP/tf-idf/" + filename

# リストに格納
with open(list_file, "r") as file:
    # ファイルの内容を行ごとに読み込む
    lines = file.readlines()  # 改行を含む行のリスト
    # 改行文字を削除してクリーンアップ
    csv_files = [line.strip() for line in lines]

# 空のリストに各CSVファイルから読み込んだデータフレームを格納
dataframes = []
file_folder='output/IP/tf-idf/'

# CSVファイルをループして読み込む
for file in csv_files:
    file_path = file_folder + file
    #print(file_path)
    if os.path.exists(file_path):  # ファイルが存在するか確認
        df = pd.read_csv(file_path)  # CSVファイルを読み込む
        dataframes.append(df)  # データフレームをリストに追加
    else:
        print(f"{file} not found.")

# 複数のデータフレームを1つに結合
combined_df = pd.concat(dataframes, ignore_index=True)

#combined_df

In [2]:
# TfidfVectorizer を作成
vectorizer = TfidfVectorizer()

# "Token_str" を使ってTF-IDF値を計算
tfidf_matrix = vectorizer.fit_transform(combined_df["token"])

# フィーチャー（トークン）の名前を取得
feature_names = vectorizer.get_feature_names_out()

# TF-IDFの値をデータフレームとして取得
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=combined_df["company_name"])

#print(tfidf_df)

In [3]:
# すべてのdftをリストに格納
#dft_list = pd.DataFrame()
dft_list = []

for tfidf_company_name in combined_df["company_name"]:

    # 各doc_nameについてTF-IDFをソート
#    dft = pd.DataFrame(tfidf_df.T[tfidf_doc_name].sort_values(ascending=False)[:25])
    dft = tfidf_df.T[tfidf_company_name].sort_values(ascending=False)[:50]
    # トークンとTF-IDF値を列として配置
    df_token_tfidf = pd.DataFrame({
        f'{tfidf_company_name}-token': dft.index,
        f'{tfidf_company_name}-tfidf': dft.values
    })
    
    # リストに追加
    dft_list.append(df_token_tfidf)

# すべてのデータフレームを列方向に結合
df_combined = pd.concat(dft_list, axis=1)

#ファイル出力時に使用する年月日文字列の取得（全処理共通）
# 現在の日付と時刻を取得
now = datetime.datetime.now()
# 年月日と時刻の文字列を生成
date_time_string = now.strftime("%Y%m%d_%H%M%S")

#ファイル出力
file_name_tfidf = f"output/IP/tf-idf/{date_time_string}_tf-idf.csv"
df_combined.to_csv(file_name_tfidf, encoding="utf_8_sig", index=False)